## Import and prep data
### 1. Import and partition the quality data

In [47]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


In [2]:
# Import ltc quality data
qual = pd.read_csv('hqo-2020-quality.csv')
# qual = qual.reset_index()
qual.info()
qual.head()
# qual.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 10 columns):
Home Name                                                     624 non-null object
Placements for referrals from all prior locations (days)◊α    624 non-null object
Placements for referrals from community (days)◊α              624 non-null object
Placements for referrals from hospital (days)◊α               624 non-null object
Antipsychotic Medication Use (%)‡*                            624 non-null object
Pressure Ulcers (%)‡*                                         624 non-null object
Falls (%)‡*                                                   624 non-null object
Physical Restraints Use (%)‡*                                 624 non-null object
Depression (%)*                                               624 non-null object
Pain (%)*                                                     624 non-null object
dtypes: object(10)
memory usage: 48.8+ KB


,Home Name,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Antipsychotic Medication Use (%)‡*,Pressure Ulcers (%)‡*,Falls (%)‡*,Physical Restraints Use (%)‡*,Depression (%)*,Pain (%)*
0,Ontario Provincial Average,147,161,90,19,2.6,16.6,3.9,22.8,5.2
1,1230839 Ontario Ltd. - Brouillette Manor,48,20,LV,28.2,5.2,33,0.4,27.9,4.7
2,2109577 Ontario Ltd. - Of Arbour Heights,166,183,55,12,3.9,15.2,8.9,30.2,3
3,412506 Ontario Ltd - St. Jacques Nursing Home,46,153,28,11.9,9.4,16.7,14.5,33.9,0
4,458422 Ontario Ltd. - Sandfield Place,334,351.5,LV,17.8,1.4,20.4,10.2,33,3.9


In [3]:
import unicodedata

# Drop row with Ontario average data
qual = qual.drop(qual.index[0])

# Clean home name 
qual['cleaned_home_name'] = qual['Home Name'].str.lower().apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
qual['cleaned_home_name'] = qual['cleaned_home_name'].map(lambda x: x.lstrip('0123456789'))
qual['cleaned_home_name'] = qual['cleaned_home_name'].str.replace(' ltc ',' long term care ')
qual['cleaned_home_name'] = qual['cleaned_home_name'].str.replace(' ltc',' long term care')

# Create 2dfs, one where cleaned_home_name contains 0 OR 1 or more hyphens
qual0 = qual[~qual['cleaned_home_name'].str.contains(' - ', regex=False, na=False)]
qual1 = qual[qual['cleaned_home_name'].str.contains(' - ', regex=False, na=False)]

print(len(qual0))
print(len(qual1))

# For qual 1 split the home name and create a new column with what comes after the hyphen
qual1['name1'] = qual['cleaned_home_name'].str.split(' - ', 1).str[1]

qual1 = qual1.reset_index()

# # There are duplicated values in qual1
# qual1['name1'].value_counts()
# # maple view 2, scarborough 2, mississauga 2

# # There are no duplicated values in qual0
# qual0['cleaned_home_name'].value_counts()

# qual.info()
# qual.head()
# qual.tail()


36
587


/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
qual1.head()

,index,Home Name,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Antipsychotic Medication Use (%)‡*,Pressure Ulcers (%)‡*,Falls (%)‡*,Physical Restraints Use (%)‡*,Depression (%)*,Pain (%)*,cleaned_home_name,name1
0,1,1230839 Ontario Ltd. - Brouillette Manor,48,20,LV,28.2,5.2,33,0.4,27.9,4.7,ontario ltd. - brouillette manor,brouillette manor
1,2,2109577 Ontario Ltd. - Of Arbour Heights,166,183,55,12,3.9,15.2,8.9,30.2,3,ontario ltd. - of arbour heights,of arbour heights
2,3,412506 Ontario Ltd - St. Jacques Nursing Home,46,153,28,11.9,9.4,16.7,14.5,33.9,0,ontario ltd - st. jacques nursing home,st. jacques nursing home
3,4,458422 Ontario Ltd. - Sandfield Place,334,351.5,LV,17.8,1.4,20.4,10.2,33,3.9,ontario ltd. - sandfield place,sandfield place
4,5,601091 Ontario Ltd. - Cedarvale Terrace,153.5,164,LV,19.1,1.2,10.7,0.9,10.3,1.2,ontario ltd. - cedarvale terrace,cedarvale terrace


In [5]:
# Review the duplicated values in qual1
# list_dup = ['maple view', 'scarborough', 'mississauga']
# Given that name1 contains 2 values for 
# 'scarborough' extendicare (canada) inc. - scarborough and hellenic home for the aged inc. - scarborough
# 'mississauga' extendicare (canada) inc. - mississauga and yee hong centre for geriatric care - mississauga
# 'maple view' extendicare (canada) inc. - maple view and southbridge care homes - maple view
# the cleaned_name column in the main data countains 
# 'scarborough' 0
# 'mississauga' 0
# 'maple view' maple view in Owen Sound

qual1[qual1['name1'].str.contains('belcrest', regex=False, na=False)]
# df_LTC_odhf[df_LTC_odhf['cleaned_name'].str.contains('belmont', regex=False, na=False)]

,index,Home Name,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Antipsychotic Medication Use (%)‡*,Pressure Ulcers (%)‡*,Falls (%)‡*,Physical Restraints Use (%)‡*,Depression (%)*,Pain (%)*,cleaned_home_name,name1
27,30,Belcrest Nursing Homes Limited - Belcrest Nursing Homes Ltd.,289,289,LV,12.9,3.5,15.3,6.7,37.3,7.6,belcrest nursing homes limited - belcrest nursing homes ltd.,belcrest nursing homes ltd.


In [6]:
## Will change the name1 for the duplicated homes in qual1 for better matching
qual1.loc[qual1['cleaned_home_name'] == 'extendicare (canada) inc. - mississauga', 'name1'] = "extendicare (canada) inc. - mississauga"
qual1.loc[qual1['cleaned_home_name'] == 'yee hong centre for geriatric care - mississauga', 'name1'] = "yee hong centre for geriatric care - mississauga"

qual1.loc[qual1['cleaned_home_name'] == 'extendicare (canada) inc. - scarborough', 'name1'] = "extendicare (canada) inc. - scarborough"
qual1.loc[qual1['cleaned_home_name'] == 'hellenic home for the aged inc. - scarborough', 'name1'] = "hellenic home for the aged inc. - scarborough"

qual1.loc[qual1['cleaned_home_name'] == 'extendicare (canada) inc. - maple view', 'name1'] = "extendicare (canada) inc. - maple view"
qual1.loc[qual1['cleaned_home_name'] == 'southbridge care homes - maple view', 'name1'] = "southbridge care homes - maple view"

## Also will change the name1 for 'drs. paul and john rekai centre' to Sherbourne Place and belcrest nursing home to belmont LTC facility
qual1.loc[qual1['name1'] == 'drs. paul and john rekai centre', 'name1'] = "sherbourne place"
qual1.loc[qual1['name1'] == 'belcrest nursing homes ltd.', 'name1'] = "belmont long term care facility"

### 2. Import the LTC data

In [7]:
df_LTC_odhf = pd.read_csv('merged_LTC_odhf.csv')
df_LTC_odhf.info()
df_LTC_odhf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 33 columns):
name                        625 non-null object
address                     625 non-null object
LHIN                        625 non-null object
home_type                   625 non-null object
short_stay                  625 non-null object
residents_council           625 non-null object
family_council              625 non-null object
accreditation               625 non-null object
additional_info             4 non-null object
city                        625 non-null object
postal_code                 625 non-null object
total_inspections           625 non-null float64
5y_inspections              625 non-null float64
2y_inspections              625 non-null float64
total_complaints            621 non-null float64
5y_complaints               588 non-null float64
2y_complaints               536 non-null float64
total_critical              620 non-null float64
5y_critical               

,name,address,LHIN,home_type,short_stay,residents_council,family_council,accreditation,additional_info,city,postal_code,total_inspections,5y_inspections,2y_inspections,total_complaints,5y_complaints,2y_complaints,total_critical,5y_critical,2y_critical,total_withOrders,5y_withOrders,2y_withOrders,cleaned_name,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,number_beds,CSDname,CSDuid,latitude,longitude
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,1200 Afton Drive,Erie St. Clair,For-Profit,No,Yes,Yes,Yes,NaN,Sarnia,N7S6L6,67.0,34.0,9.0,29.0,15.0,3.0,28.0,13.0,5.0,5.0,2.0,NaN,afton park place long term care community,NaN,NaN,NaN,NaN,128.0,Sarnia,3538030.0,42.976325,-82.361444
1,"ALBRIGHT GARDENS HOMES, INCORPORATED",5050 Hillside Drive,Hamilton Niagara Haldimand Brant (Hnhb),Non-Profit,No,Yes,Yes,No,NaN,Beamsville,L0R1B2,39.0,25.0,15.0,18.0,10.0,6.0,12.0,7.0,6.0,7.0,7.0,3.0,albright gardens,<5,0,0,Active,231.0,Lincoln,3526057.0,43.151877,-79.476598
2,ALEXANDER PLACE,329 Parkside Drive P. O. Box 50,Hamilton Niagara Haldimand Brant (Hnhb),For-Profit,Yes,Yes,Yes,Yes,NaN,Waterdown,L0R2H0,28.0,17.0,8.0,11.0,6.0,3.0,8.0,5.0,2.0,4.0,4.0,2.0,alexander place,<5,0,0,Active,128.0,Hamilton,3525005.0,43.342345,-79.900820
3,ALGOMA MANOR NURSING HOME,145 Dawson Street,North East,Non-Profit,Yes,Yes,Yes,Yes,NaN,Thessalon,P0R1L0,23.0,14.0,7.0,7.0,5.0,2.0,3.0,2.0,1.0,6.0,3.0,1.0,algoma manor nursing home,NaN,NaN,NaN,NaN,96.0,Thessalon,3557028.0,46.263462,-83.563249
4,ALGONQUIN NURSING HOME,207 Turcotte Park Road P.O. Box 270,North East,Non-Profit,Yes,Yes,Yes,Yes,NaN,Mattawa,P0H1V0,23.0,14.0,5.0,3.0,1.0,NaN,11.0,7.0,4.0,5.0,3.0,NaN,algonquin nursing home,NaN,NaN,NaN,NaN,73.0,Mattawa,3548021.0,46.313493,-78.719841


### 3. Match names from quality to main

In [8]:
# 26 homes in qual0 are also found in df_LTC_odhf, 10 not found
print(qual0['cleaned_home_name'].isin(df_LTC_odhf['cleaned_name']).value_counts())
qual0['matched'] = qual0['cleaned_home_name'].isin(df_LTC_odhf['cleaned_name'])
mask = qual0['matched'] == False
in_qual0 = qual0[mask]
in_qual0


True     26
False    10
Name: cleaned_home_name, dtype: int64


/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Home Name,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Antipsychotic Medication Use (%)‡*,Pressure Ulcers (%)‡*,Falls (%)‡*,Physical Restraints Use (%)‡*,Depression (%)*,Pain (%)*,cleaned_home_name,matched
10,Albright Gardens Homes Inc.,342,331,436.5,26,3.8,19.5,5.4,34.8,2.3,albright gardens homes inc.,False
31,Bella Senior Care Residences Inc.,91,100,29,29.2,4.5,20.9,2.2,13,7.4,bella senior care residences inc.,False
68,Centre d'accueil Roger-Séguin,112,109,153.5,11.5,1.3,16.3,6.1,30,0.3,centre d'accueil roger-seguin,False
164,Elm Grove Living Centre Inc.,131,139,LV,28.8,2,13.6,0,6.5,0.2,elm grove living centre inc.,False
250,Hellenic Care For Seniors (Toronto) Inc.,345,345,NR,8.7,0.6,3.8,0.2,8.4,0.6,hellenic care for seniors (toronto) inc.,False
291,Manitouwadge General Hospital,LV,NR,LV,43.6,2.4,9.7,ST,19.7,3.9,manitouwadge general hospital,False
325,North Renfrew LTC Services Inc.,LV,LV,NR,6.7,0,13.9,11.1,38.1,13,north renfrew long term care services inc.,False
524,Southbridge Care Home,23,30.5,LV,16,1.3,18.2,0,27.6,3.4,southbridge care home,False
612,Vision '74 Inc.,203,203,LV,12.8,2.6,15.4,4.3,21.2,2.5,vision '74 inc.,False
616,Wildwood Care Centre Inc.,93,93,LV,9,2.4,17.2,3.9,30.2,2.2,wildwood care centre inc.,False


In [9]:
from fuzzywuzzy import process

# Use fuzzy match to home names from main, with cleaned_home_name from the quality data that did not match and create a dictionary 
# with keys = cleaned_home_name from the quality data and values = matches suggested by fuzzy from the main data
in_qual0_list = list()
in_match0_list = list()

for each in in_qual0['cleaned_home_name']:
    in_qual0_list.append(each)
    highest = process.extractOne(each, df_LTC_odhf['cleaned_name'])
    in_match0_list.append(highest)

dictionary0 = dict(zip(in_qual0_list, in_match0_list))


/Users/mds/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
# Convert the dictionary to a df
df_dictionary0 = pd.DataFrame.from_dict(dictionary0, orient='index')
df_dictionary0 = df_dictionary0.reset_index()

In [11]:
# Drop unneeded columns
df_dictionary0.drop([1,2], axis = 1, inplace = True)
df_dictionary0

,index,0
0,albright gardens homes inc.,albright gardens
1,bella senior care residences inc.,bella senior care residences
2,centre d'accueil roger-seguin,centre d'accueil roger seguin
3,elm grove living centre inc.,elm grove living centre
4,hellenic care for seniors (toronto) inc.,hellenic care for seniors (toronto)
5,manitouwadge general hospital,espanola general hospital (operating as espanola nursing home-eldcap)
6,north renfrew long term care services inc.,north renfrew long-term care services inc.
7,southbridge care home,afton park place long term care community
8,vision '74 inc.,north renfrew long-term care services inc.
9,wildwood care centre inc.,wildwood care centre


In [12]:
# Where suggested fuzzy matches not correct, manually change values to the correct entry from main
# Dictionary will be used to update the quality cleaned_home_name column so that it can be merged with the main data
df_dictionary0[0].replace({"espanola general hospital (operating as espanola nursing home-eldcap)": "sante manitouwadge health"}, inplace = True)
df_dictionary0.at[8, 0] = "vision nursing home"

# Will remove southbridge care home as can not find a match in main
df_dictionary0 = df_dictionary0.drop(df_dictionary0.index[7])
df_dictionary0 = df_dictionary0.set_index('index')


In [13]:
df_dictionary0

,0
index,
albright gardens homes inc.,albright gardens
bella senior care residences inc.,bella senior care residences
centre d'accueil roger-seguin,centre d'accueil roger seguin
elm grove living centre inc.,elm grove living centre
hellenic care for seniors (toronto) inc.,hellenic care for seniors (toronto)
manitouwadge general hospital,sante manitouwadge health
north renfrew long term care services inc.,north renfrew long-term care services inc.
vision '74 inc.,vision nursing home
wildwood care centre inc.,wildwood care centre


In [14]:
# Convert back to a dictionary that we use to update the quality clean_home_name column for merging with the main
dictionary0_match = df_dictionary0.to_dict()
dictionary0_match = dictionary0_match[0]
dictionary0_match

{'albright gardens homes inc.': 'albright gardens',
 'bella senior care residences inc.': 'bella senior care residences',
 "centre d'accueil roger-seguin": "centre d'accueil roger seguin",
 'elm grove living centre inc.': 'elm grove living centre',
 'hellenic care for seniors (toronto) inc.': 'hellenic care for seniors (toronto)',
 'manitouwadge general hospital': 'sante manitouwadge health',
 'north renfrew long term care services inc.': 'north renfrew long-term care services inc.',
 "vision '74 inc.": 'vision nursing home',
 'wildwood care centre inc.': 'wildwood care centre'}

In [15]:
qual1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 13 columns):
index                                                         587 non-null int64
Home Name                                                     587 non-null object
Placements for referrals from all prior locations (days)◊α    587 non-null object
Placements for referrals from community (days)◊α              587 non-null object
Placements for referrals from hospital (days)◊α               587 non-null object
Antipsychotic Medication Use (%)‡*                            587 non-null object
Pressure Ulcers (%)‡*                                         587 non-null object
Falls (%)‡*                                                   587 non-null object
Physical Restraints Use (%)‡*                                 587 non-null object
Depression (%)*                                               587 non-null object
Pain (%)*                                                     587 non-null o

In [16]:
# Try to match name1 names from quality data with cleaned_name in the main data
# 454 homes in qual1 are also found in df_LTC_odhf, 133 not found
print(qual1['name1'].isin(df_LTC_odhf['cleaned_name']).value_counts())
qual1['matched'] = qual1['name1'].isin(df_LTC_odhf['cleaned_name'])

# Create a new df in_qual1 containing the 135 homes from quality that do not appear in main
mask = qual1['matched'] == False
in_qual1 = qual1[mask]
in_qual1.info()

True     454
False    133
Name: name1, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 1 to 586
Data columns (total 14 columns):
index                                                         133 non-null int64
Home Name                                                     133 non-null object
Placements for referrals from all prior locations (days)◊α    133 non-null object
Placements for referrals from community (days)◊α              133 non-null object
Placements for referrals from hospital (days)◊α               133 non-null object
Antipsychotic Medication Use (%)‡*                            133 non-null object
Pressure Ulcers (%)‡*                                         133 non-null object
Falls (%)‡*                                                   133 non-null object
Physical Restraints Use (%)‡*                                 133 non-null object
Depression (%)*                                               133 non-null object
Pain (%)*               

In [17]:
# Fuzzy match home names from main with name1 from the quality data
in_qual1_list = list()
in_match1_list = list()

for each in in_qual1['name1']:
    in_qual1_list.append(each)
    highest = process.extractOne(each, df_LTC_odhf['cleaned_name'])
    in_match1_list.append(highest)

dictionary1 = dict(zip(in_qual1_list, in_match1_list))

In [18]:
# Convert the dictionary to a df
df_dictionary1 = pd.DataFrame.from_dict(dictionary1, orient='index')
df_dictionary1 = df_dictionary1.reset_index()

# Drop unneeded columns
df_dictionary1.drop([1,2], axis = 1, inplace = True)
df_dictionary1.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
index    133 non-null object
0        133 non-null object
dtypes: object(2)
memory usage: 2.2+ KB


In [19]:
df_dictionary1

,index,0
0,of arbour heights,arbour heights
1,nipissing manor nursing care centre,nipissing manor nursing care center
2,valley view residence,valleyview residence
3,algonquin nursing home of mattawa ltd.,algonquin nursing home
4,centennial place long term care home,centennial place long-term care home
5,moira place long term care home,moira place long-term care home
6,bayhaven nursing home,bay haven nursing home
7,bluewater rest home,blue water rest home
8,pinecrest,pinecrest (kenora)
9,brucelea haven long term care home,brucelea haven long term care home - corporation of the county of bruce


In [20]:
# df_LTC_odhf[df_LTC_odhf['cleaned_name'].str.contains('meaford', regex=False, na=False)]
# qual[qual['cleaned_home_name'].str.contains('meaford', regex=False, na=False)]

In [21]:
# Manually review and remove homes without a match in main
df_dictionary1 = df_dictionary1.drop(df_dictionary1.index[[95]])
'veterans beds'

df_dictionary1 = df_dictionary1.reset_index()
df_dictionary1.drop(['level_0'], axis = 1, inplace = True)
df_dictionary1

,index,0
0,of arbour heights,arbour heights
1,nipissing manor nursing care centre,nipissing manor nursing care center
2,valley view residence,valleyview residence
3,algonquin nursing home of mattawa ltd.,algonquin nursing home
4,centennial place long term care home,centennial place long-term care home
5,moira place long term care home,moira place long-term care home
6,bayhaven nursing home,bay haven nursing home
7,bluewater rest home,blue water rest home
8,pinecrest,pinecrest (kenora)
9,brucelea haven long term care home,brucelea haven long term care home - corporation of the county of bruce


In [22]:
# Replace values for incorrect fuzzy matches
df_dictionary1.loc[df_dictionary1['index'] == 'elmira term care residence', 0] = "chartwell elmira long term care residence"
df_dictionary1.loc[df_dictionary1['index'] == 'country village health care centre', 0] = "country village homes - woodslee"
df_dictionary1.loc[df_dictionary1['index'] == 'london', 0] = "extendicare london"
df_dictionary1.loc[df_dictionary1['index'] == 'hilltop manor nursing home limited', 0] = "hilltop manor nursing home (merrickville)"
df_dictionary1.loc[df_dictionary1['index'] == 'meaford long term care centre', 0] = "meaford long term care - a peoplecare community"
df_dictionary1.loc[df_dictionary1['index'] == 'victoria gardens ltc', 0] = "victoria gardens long term care"
df_dictionary1.loc[df_dictionary1['index'] == 'long term care unit', 0] = "north shore health network - ltc unit"
df_dictionary1.loc[df_dictionary1['index'] == 'peoplecare ar goudie', 0] = "peoplecare a.r. goudie kitchener"
df_dictionary1.loc[df_dictionary1['index'] == 'standard beds', 0] = "the perley and rideau veterans' health centre"
df_dictionary1.loc[df_dictionary1['index'] == 'oa pleasant manor', 0] = "pleasant manor retirement village"
df_dictionary1.loc[df_dictionary1['index'] == 'errinrung nursing home', 0] = "errinrung long term care home"
df_dictionary1.loc[df_dictionary1['index'] == 'queensway nursing home', 0] = "queensway long term care home"
df_dictionary1.loc[df_dictionary1['index'] == 'regency manor nursing home', 0] = "regency long term care home"
df_dictionary1.loc[df_dictionary1['index'] == 'shelburne nursing home', 0] = "shelburne long term care home"
df_dictionary1.loc[df_dictionary1['index'] == 'lakehead manor', 0] = "southbridge lakehead"
df_dictionary1.loc[df_dictionary1['index'] == 'pinewood court', 0] = "southbridge pinewood"
df_dictionary1.loc[df_dictionary1['index'] == 'the elliott community', 0] = "the elliott long term care residence"
df_dictionary1.loc[df_dictionary1['index'] == 'parkview home ltc', 0] = "parkview home long-term care"
df_dictionary1.loc[df_dictionary1['index'] == 'vera m. davis nursing home', 0] = "vera m. davis community care centre"
df_dictionary1.loc[df_dictionary1['index'] == 'marnwood lifecare centre', 0] = "glen hill marnwood"
df_dictionary1.loc[df_dictionary1['index'] == 'woodstock', 0] = "woodingford lodge - woodstock"
df_dictionary1.loc[df_dictionary1['index'] == 'pinecrest nursing home', 0] = "pinecrest (plantagenet)"
df_dictionary1.loc[df_dictionary1['index'] == 'saugeen valley nursing center', 0] = "strathcona long term care"
df_dictionary1.loc[df_dictionary1['index'] == '', 0] = "rose of sharon korean long term care"
df_dictionary1.loc[df_dictionary1['index'] == 'extendicare (canada) inc. - maple view', 0] = "extendicare maple view of sault ste. marie"
df_dictionary1.loc[df_dictionary1['index'] == 'extendicare (canada) inc. - mississauga', 0] = "extendicare mississauga"
df_dictionary1.loc[df_dictionary1['index'] == 'extendicare (canada) inc. - scarborough', 0] = "extendicare scarborough"
df_dictionary1.loc[df_dictionary1['index'] == 'hellenic home for the aged inc. - scarborough', 0] = "hellenic home - scarborough"
df_dictionary1.loc[df_dictionary1['index'] == 'odd fellow and rebekah home', 0] = "ioof seniors home"
df_dictionary1.loc[df_dictionary1['index'] == 'yee hong centre for geriatric care - mississauga', 0] = "yee hong centre - mississauga"
df_dictionary1.loc[df_dictionary1['index'] == 'espanola nursing home', 0] = "espanola general hospital (operating as espanola nursing home-ltc)"

df_dictionary1 = df_dictionary1.set_index('index')
df_dictionary1

# Convert back to a dictionary
dictionary1_match = df_dictionary1.to_dict()
dictionary1_match = dictionary1_match[0]
dictionary1_match

{'of arbour heights': 'arbour heights',
 'nipissing manor nursing care centre': 'nipissing manor nursing care center',
 'valley view residence': 'valleyview residence',
 'algonquin nursing home of mattawa ltd.': 'algonquin nursing home',
 'centennial place long term care home': 'centennial place long-term care home',
 'moira place long term care home': 'moira place long-term care home',
 'bayhaven nursing home': 'bay haven nursing home',
 'bluewater rest home': 'blue water rest home',
 'pinecrest': 'pinecrest (kenora)',
 'brucelea haven long term care home': 'brucelea haven long term care home - corporation of the county of bruce',
 'elisabeth brueyre residence': 'elisabeth-bruyere residence',
 'arthur nursing home': 'caressant care arthur nursing home',
 'bourget': 'caressant care bourget',
 'cobden': 'caressant care - cobden',
 'courtland': 'caressant care courtland',
 'fergus nursing home': 'caressant care fergus nursing home',
 'harriston': 'caressant care - harriston',
 'lindsay n

In [23]:
# df_dictionary1[df_dictionary1['index'].str.contains('', regex=False, na=False)]
# df_dictionary1[df_dictionary1['index'] == '']
# qual1[qual1['name1'].str.contains('scarborough', regex=False, na=False)]

### Manual review
- "perley and rideau veterans' health centre - standard beds" and "perley and rideau veterans' health centre - veterans beds" listed as one home in main
    - "the perley and rideau veterans' health centre"
    - some of the veterans quality data is missing so will keep the standard
- In quality data there was one home where nothing came after the hyphen, rose of sharon (ontario) retirement community - 

In [24]:
# Check for duplicates in dictionary1
dictionary1_keys = []
dictionary1_values = []
for k,v in dictionary1_match.items():
    dictionary1_keys.append(k)
    dictionary1_values.append(v)
# dictionary1_keys[0:5]
# dictionary1_values[0:5]

# There are duplicate values in dictionary1
print(len(dictionary1_values))
print(len(set(dictionary1_values)))


132
132


In [25]:
# Combine the 2 dictionaries
# len(dictionary0_match) + len(dictionary1_match)
dictionary_match = {**dictionary0_match, **dictionary1_match}
print(len(dictionary_match))
print(len(set(dictionary_match)))

141
141


In [26]:
# Copy the cleaned_home_name column into a name1 column for matching
qual0['name1'] = qual0['cleaned_home_name']

/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
qual0

,Home Name,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Antipsychotic Medication Use (%)‡*,Pressure Ulcers (%)‡*,Falls (%)‡*,Physical Restraints Use (%)‡*,Depression (%)*,Pain (%)*,cleaned_home_name,matched,name1
10,Albright Gardens Homes Inc.,342,331,436.5,26,3.8,19.5,5.4,34.8,2.3,albright gardens homes inc.,False,albright gardens homes inc.
21,Atikokan General Hospital,261,LV,261,41.4,8.4,14.2,9,42.4,11.2,atikokan general hospital,True,atikokan general hospital
31,Bella Senior Care Residences Inc.,91,100,29,29.2,4.5,20.9,2.2,13,7.4,bella senior care residences inc.,False,bella senior care residences inc.
33,Berkshire Care Centre,32.5,34,26,21.2,3.9,15.8,6.4,16.8,3.8,berkshire care centre,True,berkshire care centre
68,Centre d'accueil Roger-Séguin,112,109,153.5,11.5,1.3,16.3,6.1,30,0.3,centre d'accueil roger-seguin,False,centre d'accueil roger-seguin
108,Copernicus Lodge,275,279,LV,29.8,2.1,10.5,4.9,23,2.1,copernicus lodge,True,copernicus lodge
164,Elm Grove Living Centre Inc.,131,139,LV,28.8,2,13.6,0,6.5,0.2,elm grove living centre inc.,False,elm grove living centre inc.
235,Fairview Mennonite Home,110,137,LV,16.1,3.1,21.6,2.9,47,2.8,fairview mennonite home,True,fairview mennonite home
242,Geraldton District Hospital,500,LV,504,62.8,1.4,5.2,8.8,7.5,48,geraldton district hospital,True,geraldton district hospital
246,Grove Park Home for Senior Citizens,226,226,LV,12.3,4.1,18.3,1.5,30,6.4,grove park home for senior citizens,True,grove park home for senior citizens


In [28]:
# Combine the qual0 and qual1 dfs
qual_full = pd.concat([qual0, qual1])

qual_full.info()
qual_full.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 10 to 586
Data columns (total 14 columns):
Antipsychotic Medication Use (%)‡*                            623 non-null object
Depression (%)*                                               623 non-null object
Falls (%)‡*                                                   623 non-null object
Home Name                                                     623 non-null object
Pain (%)*                                                     623 non-null object
Physical Restraints Use (%)‡*                                 623 non-null object
Placements for referrals from all prior locations (days)◊α    623 non-null object
Placements for referrals from community (days)◊α              623 non-null object
Placements for referrals from hospital (days)◊α               623 non-null object
Pressure Ulcers (%)‡*                                         623 non-null object
cleaned_home_name                                             623 non-null

/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Antipsychotic Medication Use (%)‡*,Depression (%)*,Falls (%)‡*,Home Name,Pain (%)*,Physical Restraints Use (%)‡*,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Pressure Ulcers (%)‡*,cleaned_home_name,index,matched,name1
10,26,34.8,19.5,Albright Gardens Homes Inc.,2.3,5.4,342,331,436.5,3.8,albright gardens homes inc.,NaN,False,albright gardens homes inc.
21,41.4,42.4,14.2,Atikokan General Hospital,11.2,9,261,LV,261,8.4,atikokan general hospital,NaN,True,atikokan general hospital
31,29.2,13,20.9,Bella Senior Care Residences Inc.,7.4,2.2,91,100,29,4.5,bella senior care residences inc.,NaN,False,bella senior care residences inc.
33,21.2,16.8,15.8,Berkshire Care Centre,3.8,6.4,32.5,34,26,3.9,berkshire care centre,NaN,True,berkshire care centre
68,11.5,30,16.3,Centre d'accueil Roger-Séguin,0.3,6.1,112,109,153.5,1.3,centre d'accueil roger-seguin,NaN,False,centre d'accueil roger-seguin


In [29]:
# Using the dictionary replace name1 with names to match those in main
qual_full["name1"].replace(dictionary_match, inplace=True)

### Merge quality data with main data

In [118]:
df_all = pd.merge(right=df_LTC_odhf, left=qual_full, 
                  how='outer', right_on='cleaned_name', 
                  left_on='name1')

# 627 entries, 623 with quality data, 4 homes without quality data
df_all.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 627 entries, 0 to 626
Data columns (total 47 columns):
Antipsychotic Medication Use (%)‡*                            623 non-null object
Depression (%)*                                               623 non-null object
Falls (%)‡*                                                   623 non-null object
Home Name                                                     623 non-null object
Pain (%)*                                                     623 non-null object
Physical Restraints Use (%)‡*                                 623 non-null object
Placements for referrals from all prior locations (days)◊α    623 non-null object
Placements for referrals from community (days)◊α              623 non-null object
Placements for referrals from hospital (days)◊α               623 non-null object
Pressure Ulcers (%)‡*                                         623 non-null object
cleaned_home_name                                             623 non-null 

In [119]:
# # Code to investigate homes in quality
# qual_full[qual_full['name1'].str.contains('belmont', regex=False, na=False)]['name1']
 

### Manual review 
Elderly Capital Assistance Program (ELDCAP) provides services to long-term care residents
in units that are collocated within hospitals in small northern communities. ELDCAP beds are
subject to the long-term care program requirements but are funded through a hospital's global
budget. ELDCAP beds are also used to classify interim long-term care beds opened temporarily
in hospitals. 

The following ELDCAP units where not represented in the quality data:

1. lakeland long term care (eldcap) (20 beds)
    - west parry sound health centre lakeland long term care services corporation in the quality data was matched to lakeland long term care services (80 beds) 
2. north shore health network – eldcap (10 beds) 
    - north shore health network - long term care unit in the quality data was matched to north shore health network – ltc unit in main	(22 beds)
3. espanola general hospital (operating as espanola nursing home-eldcap) (30 beds)
    - espanola general hospital - espanola nursing home in the quality data was matched to espanola general hospital (operating as espanola nursing home-ltc) (32 beds)


The following in main was not in quality:
- seaforth long term care home

Will remove homes that are not represented in the quality data. 

In [120]:
# Remove 4 homes without quality data
df_all.drop(df_all.index[df_all['cleaned_name'] == 'seaforth long term care home'], inplace = True)
df_all.drop(df_all.index[df_all['cleaned_name'] == 'north shore health network - eldcap unit'], inplace = True)
df_all.drop(df_all.index[df_all['cleaned_name'] == 'lakeland long term care (eldcap)'], inplace = True)
df_all.drop(df_all.index[df_all['cleaned_name'] == 'espanola general hospital (operating as espanola nursing home-eldcap)'], inplace = True)
df_all[df_all['name'].isnull()]


,Antipsychotic Medication Use (%)‡*,Depression (%)*,Falls (%)‡*,Home Name,Pain (%)*,Physical Restraints Use (%)‡*,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Pressure Ulcers (%)‡*,cleaned_home_name,index,matched,name1,name,address,LHIN,home_type,short_stay,residents_council,family_council,accreditation,additional_info,city,postal_code,total_inspections,5y_inspections,2y_inspections,total_complaints,5y_complaints,2y_complaints,total_critical,5y_critical,2y_critical,total_withOrders,5y_withOrders,2y_withOrders,cleaned_name,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,number_beds,CSDname,CSDuid,latitude,longitude
30,16,27.6,18.2,Southbridge Care Home,3.4,0,23,30.5,LV,1.3,southbridge care home,NaN,False,southbridge care home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,13,34.7,18.2,Perley and Rideau Veterans' Health Centre - Veterans Beds,11.6,8.3,NR,NR,NR,3.1,perley and rideau veterans' health centre - veterans beds,356.0,False,veterans beds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# Review homes present in the quality data but not in the main
df_all[df_all['name'].isnull()]


,Antipsychotic Medication Use (%)‡*,Depression (%)*,Falls (%)‡*,Home Name,Pain (%)*,Physical Restraints Use (%)‡*,Placements for referrals from all prior locations (days)◊α,Placements for referrals from community (days)◊α,Placements for referrals from hospital (days)◊α,Pressure Ulcers (%)‡*,cleaned_home_name,index,matched,name1,name,address,LHIN,home_type,short_stay,residents_council,family_council,accreditation,additional_info,city,postal_code,total_inspections,5y_inspections,2y_inspections,total_complaints,5y_complaints,2y_complaints,total_critical,5y_critical,2y_critical,total_withOrders,5y_withOrders,2y_withOrders,cleaned_name,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,number_beds,CSDname,CSDuid,latitude,longitude
30,16,27.6,18.2,Southbridge Care Home,3.4,0,23,30.5,LV,1.3,southbridge care home,NaN,False,southbridge care home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,13,34.7,18.2,Perley and Rideau Veterans' Health Centre - Veterans Beds,11.6,8.3,NR,NR,NR,3.1,perley and rideau veterans' health centre - veterans beds,356.0,False,veterans beds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# Remove 2 homes with quality data that are not in the main
# Southbridge Care Home can not be matched to anything in main
# Perley and Rideau Veterans' Health Centre - Veterans Beds, already matched to standard beds

df_all.drop(df_all.index[df_all['Home Name'] == 'Southbridge Care Home'], inplace = True)
df_all.drop(df_all.index[df_all['Home Name'] == 'Perley and Rideau Veterans\' Health Centre - Veterans Beds'], inplace = True)

In [123]:
# Clean up columns
listofcols = ['Home Name', 'cleaned_home_name', 'index', 'matched', 'name1', 'additional_info']
df_all.drop(listofcols, axis = 1, inplace = True)

In [124]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 0 to 622
Data columns (total 41 columns):
Antipsychotic Medication Use (%)‡*                            621 non-null object
Depression (%)*                                               621 non-null object
Falls (%)‡*                                                   621 non-null object
Pain (%)*                                                     621 non-null object
Physical Restraints Use (%)‡*                                 621 non-null object
Placements for referrals from all prior locations (days)◊α    621 non-null object
Placements for referrals from community (days)◊α              621 non-null object
Placements for referrals from hospital (days)◊α               621 non-null object
Pressure Ulcers (%)‡*                                         621 non-null object
name                                                          621 non-null object
address                                                       621 non-null 

In [125]:
# Rename columns
df_all.rename(columns={'Antipsychotic Medication Use (%)‡*':'antipsychotic_percent',
                   'Depression (%)*':'depression_percent',
                   'Falls (%)‡*':'falls_percent',
                   'Pain (%)*':'pain_percent',
                   'Physical Restraints Use (%)‡*':'restraints_percent',
                   'Placements for referrals from all prior locations (days)◊α':'placements_all_days',
                   'Placements for referrals from community (days)◊α': 'placements_community_days',
                   'Placements for referrals from hospital (days)◊α':'placements_hospital_days',
                   'Pressure Ulcers (%)‡*':'pressure_ulcers_percent',
                   'Confirmed Resident Cases':'confirmed_resident_cases',
                   'Resident Deaths':'resident_deaths',
                   'Confirmed Staff Cases':'confirmed_staff_cases',
                   'Status':'status'}, inplace=True)

In [126]:
df_all.columns

Index(['antipsychotic_percent', 'depression_percent', 'falls_percent',
       'pain_percent', 'restraints_percent', 'placements_all_days',
       'placements_community_days', 'placements_hospital_days',
       'pressure_ulcers_percent', 'name', 'address', 'LHIN', 'home_type',
       'short_stay', 'residents_council', 'family_council', 'accreditation',
       'city', 'postal_code', 'total_inspections', '5y_inspections',
       '2y_inspections', 'total_complaints', '5y_complaints', '2y_complaints',
       'total_critical', '5y_critical', '2y_critical', 'total_withOrders',
       '5y_withOrders', '2y_withOrders', 'cleaned_name',
       'confirmed_resident_cases', 'resident_deaths', 'confirmed_staff_cases',
       'status', 'number_beds', 'CSDname', 'CSDuid', 'latitude', 'longitude'],
      dtype='object')

In [127]:
df_all = df_all[['name', 'cleaned_name', 'address', 'LHIN', 'home_type', 'city', 'postal_code',
        'short_stay', 'residents_council', 'family_council', 'accreditation',
        'total_inspections','5y_inspections', '2y_inspections', 'total_complaints', '5y_complaints',
        '2y_complaints', 'total_critical', '5y_critical', '2y_critical',
        'total_withOrders', '5y_withOrders', '2y_withOrders',
        'antipsychotic_percent', 'depression_percent', 'falls_percent', 
        'pressure_ulcers_percent', 'pain_percent', 'restraints_percent', 
        'placements_all_days', 'placements_community_days', 'placements_hospital_days',
        'confirmed_resident_cases', 'resident_deaths', 'confirmed_staff_cases',
        'status', 'number_beds', 'CSDname', 'CSDuid', 'latitude', 'longitude']]

In [128]:
listofcols = ['placements_all_days', 'placements_community_days', 'placements_hospital_days']
df_all.drop(listofcols, axis = 1, inplace = True)

In [129]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 0 to 622
Data columns (total 38 columns):
name                        621 non-null object
cleaned_name                621 non-null object
address                     621 non-null object
LHIN                        621 non-null object
home_type                   621 non-null object
city                        621 non-null object
postal_code                 621 non-null object
short_stay                  621 non-null object
residents_council           621 non-null object
family_council              621 non-null object
accreditation               621 non-null object
total_inspections           621 non-null float64
5y_inspections              621 non-null float64
2y_inspections              621 non-null float64
total_complaints            618 non-null float64
5y_complaints               586 non-null float64
2y_complaints               535 non-null float64
total_critical              616 non-null float64
5y_critical             

,name,cleaned_name,address,LHIN,home_type,city,postal_code,short_stay,residents_council,family_council,accreditation,total_inspections,5y_inspections,2y_inspections,total_complaints,5y_complaints,2y_complaints,total_critical,5y_critical,2y_critical,total_withOrders,5y_withOrders,2y_withOrders,antipsychotic_percent,depression_percent,falls_percent,pressure_ulcers_percent,pain_percent,restraints_percent,confirmed_resident_cases,resident_deaths,confirmed_staff_cases,status,number_beds,CSDname,CSDuid,latitude,longitude
0,"ALBRIGHT GARDENS HOMES, INCORPORATED",albright gardens,5050 Hillside Drive,Hamilton Niagara Haldimand Brant (Hnhb),Non-Profit,Beamsville,L0R1B2,No,Yes,Yes,No,39.0,25.0,15.0,18.0,10.0,6.0,12.0,7.0,6.0,7.0,7.0,3.0,26,34.8,19.5,3.8,2.3,5.4,<5,0,0,Active,231.0,Lincoln,3526057.0,43.151877,-79.476598
1,ATIKOKAN GENERAL HOSPITAL,atikokan general hospital,120 Dorothy Street,North West,Non-Profit,Atikokan,P0T1C0,No,No,Yes,Yes,11.0,7.0,4.0,1.0,NaN,NaN,2.0,1.0,1.0,3.0,2.0,1.0,41.4,42.4,14.2,8.4,11.2,9,NaN,NaN,NaN,NaN,26.0,Atikokan,3559001.0,48.754155,-91.597609
2,BELLA SENIOR CARE RESIDENCES INC.,bella senior care residences,8720 Willoughby Drive,Hamilton Niagara Haldimand Brant (Hnhb),For-Profit,Niagara Falls,L2G7X3,No,Yes,Yes,Yes,67.0,46.0,13.0,35.0,22.0,5.0,12.0,8.0,4.0,18.0,12.0,3.0,29.2,13,20.9,4.5,7.4,2.2,NaN,NaN,NaN,NaN,161.0,Niagara Falls,3526043.0,43.050842,-79.045438
3,BERKSHIRE CARE CENTRE,berkshire care centre,350 Dougall Avenue,Erie St. Clair,For-Profit,Windsor,N9A4P4,No,Yes,Yes,Yes,172.0,72.0,25.0,94.0,34.0,9.0,59.0,26.0,12.0,12.0,8.0,4.0,21.2,16.8,15.8,3.9,3.8,6.4,NaN,0,NaN,Inactive,231.0,Windsor,3537039.0,42.316316,-83.041620
4,CENTRE D'ACCUEIL ROGER SEGUIN,centre d'accueil roger seguin,435 Lemay Street,Champlain,Non-Profit,Clarence Creek,K0A1N0,Yes,Yes,Yes,No,51.0,30.0,20.0,8.0,2.0,NaN,19.0,14.0,14.0,16.0,8.0,4.0,11.5,30,16.3,1.3,0.3,6.1,NaN,0,NaN,Inactive,113.0,Clarence-Rockland,3502036.0,45.510257,-75.216169


In [130]:
df_all.columns

Index(['name', 'cleaned_name', 'address', 'LHIN', 'home_type', 'city',
       'postal_code', 'short_stay', 'residents_council', 'family_council',
       'accreditation', 'total_inspections', '5y_inspections',
       '2y_inspections', 'total_complaints', '5y_complaints', '2y_complaints',
       'total_critical', '5y_critical', '2y_critical', 'total_withOrders',
       '5y_withOrders', '2y_withOrders', 'antipsychotic_percent',
       'depression_percent', 'falls_percent', 'pressure_ulcers_percent',
       'pain_percent', 'restraints_percent', 'confirmed_resident_cases',
       'resident_deaths', 'confirmed_staff_cases', 'status', 'number_beds',
       'CSDname', 'CSDuid', 'latitude', 'longitude'],
      dtype='object')

In [131]:
# Replace NaN values with 0 for 'total_complaints', '5y_complaints', '2y_complaints',
# 'total_critical', '5y_critical', '2y_critical', 'total_withOrders',
# '5y_withOrders', '2y_withOrders'

listofcol = ['total_complaints', '5y_complaints', '2y_complaints',
             'total_critical', '5y_critical', '2y_critical', 'total_withOrders',
             '5y_withOrders', '2y_withOrders']
for each in listofcol:
    df_all[each].fillna(0, inplace = True)

A few homes are missing precise values for quality measures:
ST: Data point is not precise
NR: Facility is not required to report

Will remove these homes. 

In [137]:
# List homes where quality measures not reported, 6 homes with this issue
listofcol = ['antipsychotic_percent',
             'depression_percent', 'falls_percent', 'pressure_ulcers_percent',
             'pain_percent', 'restraints_percent']

values = ['ST', 'NR']

for each in listofcol:
    df_removed = pd.concat([df_removed, df_all[df_all[each].isin(values)]])

missing_qual = set(df_removed['name'])

In [138]:
# Remove homes with incomplete quality data, 615 homes left
for each in missing_qual:
    df_all.drop(df_all.index[df_all['name'] == each], inplace = True)

df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 622
Data columns (total 38 columns):
name                        615 non-null object
cleaned_name                615 non-null object
address                     615 non-null object
LHIN                        615 non-null object
home_type                   615 non-null object
city                        615 non-null object
postal_code                 615 non-null object
short_stay                  615 non-null object
residents_council           615 non-null object
family_council              615 non-null object
accreditation               615 non-null object
total_inspections           615 non-null float64
5y_inspections              615 non-null float64
2y_inspections              615 non-null float64
total_complaints            615 non-null float64
5y_complaints               615 non-null float64
2y_complaints               615 non-null float64
total_critical              615 non-null float64
5y_critical             

In [139]:
# Change the datatype to numeric for 'antipsychotic_percent',
# 'depression_percent', 'falls_percent', 'pressure_ulcers_percent',
# 'pain_percent', 'restraints_percent'

listofcol = ['antipsychotic_percent',
             'depression_percent', 'falls_percent', 'pressure_ulcers_percent',
             'pain_percent', 'restraints_percent']

for each in listofcol:
    df_all[each] = pd.to_numeric(df_all[each])

In [140]:
df_all.to_csv(r'merged_LTC_odhf_quality.csv', index = False)

In [ ]:
# # Code to investigate homes in quality
# qual[qual['cleaned_home_name'].str.contains('seaforth', regex=False, na=False)]
# # for each in qual['cleaned_home_name']:
# #     if each.endswith('- '):
# #         print(each)

In [ ]:
# # Code to investigate homes in main
# df_LTC_odhf[df_LTC_odhf['cleaned_name'].str.contains('seaforth', regex=False, na=False)]
# # # df_trim[df_trim['cleaned_name'].str.contains('lakeland', regex=False, na=False)]['cleaned_name']
